In [2]:
import json
import math
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm import tqdm

# ----- Load data with sentiment vector -----
def load_rating_with_sentiment(path):
    rows = []
    with open(path, encoding="utf-8") as f:
        for line in f:
            d = json.loads(line)
            rows.append({
                "user": d["user_id"],
                "biz": d["business_id"],
                "stars": d["stars"],
                "sentiment_vector": np.array(d["sentiment_vector"])
            })
    return pd.DataFrame(rows)

# ----- Leave-One-Out split -----
def leave_one_out(ratings):
    np.random.seed(42)
    train_idx = []
    test = {}
    for u, grp in ratings.groupby("user"):
        idx = grp.index.values
        if len(idx) > 1:
            hold = np.random.choice(idx)
            test[u] = ratings.loc[hold, "biz"]
            train_idx.extend([i for i in idx if i != hold])
        else:
            train_idx.extend(idx)
    return ratings.loc[train_idx].reset_index(drop=True), test

# ----- Build mappings -----
def build_maps(train_df):
    item_users = defaultdict(dict)
    user_items = defaultdict(dict)
    item_sentiments = defaultdict(list)
    for _, r in train_df.iterrows():
        item_users[r["biz"]][r["user"]] = r["stars"]
        user_items[r["user"]][r["biz"]] = r["stars"]
        item_sentiments[r["biz"]].append(r["sentiment_vector"])
    # 평균 벡터 계산
    item_sentiment_avg = {biz: np.mean(vectors, axis=0) for biz, vectors in item_sentiments.items()}
    return item_users, user_items, item_sentiment_avg

# ----- Cosine similarity -----
def cosine_similarity_dict(a, b):
    common = set(a) & set(b)
    if not common:
        return 0.0
    num = sum(a[u] * b[u] for u in common)
    den = (
        math.sqrt(sum(v * v for v in a.values())) *
        math.sqrt(sum(v * v for v in b.values())) + 1e-9
    )
    return num / den

def cosine_similarity_vec(a, b):
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0.0
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

# ----- Precompute hybrid item similarities -----
def precompute_hybrid_sims(item_users, item_sentiment_avg, alpha=0.9):
    items = list(item_users)
    sims = defaultdict(dict)

    for i, a in tqdm(enumerate(items), total=len(items), desc="하이브리드 유사도 계산"):
        for b in items[i + 1:]:
            rating_sim = cosine_similarity_dict(item_users[a], item_users[b])
            sent_sim = cosine_similarity_vec(
                item_sentiment_avg.get(a, np.zeros(15)),
                item_sentiment_avg.get(b, np.zeros(15))
            )
            hybrid_sim = alpha * rating_sim + (1 - alpha) * sent_sim
            if hybrid_sim > 0:
                sims[a][b] = hybrid_sim
                sims[b][a] = hybrid_sim
    return sims

# ----- Recommendation -----
def recommend(user, user_items, item_sims, n=5):
    seen = set(user_items[user].keys())
    scores = defaultdict(float)
    for item, rating in user_items[user].items():
        for similar_item, sim in item_sims.get(item, {}).items():
            if similar_item not in seen:
                scores[similar_item] += sim * rating
    return [item for item, _ in sorted(scores.items(), key=lambda x: -x[1])[:n]]

# ----- Evaluation -----
def precision_at_k(test, recs, k=5):
    return sum(1 for u, gt in test.items() if gt in recs.get(u, [])[:k]) / len(test)

def recall_at_k(test, recs, k=5):
    return precision_at_k(test, recs, k)

def ndcg_at_k(test, recs, k=5):
    total = 0.0
    for u, gt in test.items():
        rec_list = recs.get(u, [])[:k]
        if gt in rec_list:
            idx = rec_list.index(gt)
            total += 1 / math.log2(idx + 2)
    return total / len(test)

def hr_at_k(test, recs, k=5):
    """Hit Rate@K 계산"""
    hits = 0
    for u, test_items in test.items():
        rec_list = recs.get(u, [])[:k]
        if any(item in rec_list for item in test_items):
            hits += 1
    return hits / len(test)

# ================================
# 실행 코드 (Jupyter 셀에서 사용)
# ================================
rating_path = "review_business_5up_5aspect_3sentiment_vectorized_clean.json"
topn = 5
min_ratings = 5

print(f"데이터 로드 중: {rating_path}")
ratings = load_rating_with_sentiment(rating_path)

user_counts = ratings.groupby("user").size()
valid_users = user_counts[user_counts >= min_ratings].index
ratings = ratings[ratings["user"].isin(valid_users)]

print(f"총 {len(ratings):,}개 평점, {ratings['user'].nunique():,}명 사용자, {ratings['biz'].nunique():,}개 아이템")

print("학습/테스트 데이터 분리 중...")
train, test = leave_one_out(ratings)
print(f"학습: {len(train):,}개, 테스트: {len(test):,}개")

print("맵 및 감성 벡터 평균 계산 중...")
item_users, user_items, item_sentiment_avg = build_maps(train)

# 평가 지표를 기록할 리스트 초기화
results = []
# alpha 값에 대해 테스트 (0.00 ~ 1.00, 0.05 간격)
for alpha in np.arange(0.00, 1.01, 0.05):
    print(f"alpha = {alpha:.2f}")
    print("하이브리드 유사도 계산 중...")
    item_sims = precompute_hybrid_sims(item_users, item_sentiment_avg, alpha=alpha)

    print(f"상위 {topn}개 아이템 추천 중...")
    recommendations = {}
    for user in tqdm(test.keys()):
        if user in user_items and len(user_items[user]) > 0:
            recommendations[user] = recommend(user, user_items, item_sims, topn)

    print("성능 평가 중...")
    p5 = precision_at_k(test, recommendations, 5)
    r5 = recall_at_k(test, recommendations, 5)
    n5 = ndcg_at_k(test, recommendations, 5)
    hr5 = hr_at_k(test, recommendations, 5)

    results.append({
        "alpha": alpha,
        "Precision@5": p5,
        "Recall@5": r5,
        "NDCG@5": n5,
        "HR@5": hr5
    })

# 성능 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 각 지표별로 최고의 alpha 값을 찾아 출력
best_precision = results_df.loc[results_df['Precision@5'].idxmax()]
best_recall = results_df.loc[results_df['Recall@5'].idxmax()]
best_ndcg = results_df.loc[results_df['NDCG@5'].idxmax()]
best_hr = results_df.loc[results_df['HR@5'].idxmax()]

print("\n최고 성능 (Precision@5 기준):")
print(best_precision)

print("\n최고 성능 (Recall@5 기준):")
print(best_recall)

print("\n최고 성능 (NDCG@5 기준):")
print(best_ndcg)

print("\n최고 성능 (HR@5 기준):")
print(best_hr)


데이터 로드 중: review_business_5up_5aspect_3sentiment_vectorized_clean.json
총 447,796개 평점, 27,807명 사용자, 6,831개 아이템
학습/테스트 데이터 분리 중...
학습: 419,989개, 테스트: 27,807개
맵 및 감성 벡터 평균 계산 중...
alpha = 0.00
하이브리드 유사도 계산 중...


하이브리드 유사도 계산: 100%|██████████| 6819/6819 [04:35<00:00, 24.78it/s] 


상위 5개 아이템 추천 중...


100%|██████████| 27807/27807 [09:05<00:00, 50.97it/s]


성능 평가 중...
alpha = 0.05
하이브리드 유사도 계산 중...


...

성능 평가 중...
alpha = 1.00
하이브리드 유사도 계산 중...


하이브리드 유사도 계산: 100%|██████████| 6819/6819 [04:24<00:00, 25.74it/s] 


상위 5개 아이템 추천 중...


100%|██████████| 27807/27807 [04:20<00:00, 106.82it/s]


성능 평가 중...

최고 성능 (Precision@5 기준):
alpha          0.950000
Precision@5    0.056425
Recall@5       0.056425
NDCG@5         0.038033
HR@5           0.000000
Name: 19, dtype: float64

최고 성능 (Recall@5 기준):
alpha          0.950000
Precision@5    0.056425
Recall@5       0.056425
NDCG@5         0.038033
HR@5           0.000000
Name: 19, dtype: float64

최고 성능 (NDCG@5 기준):
alpha          0.950000
Precision@5    0.056425
Recall@5       0.056425
NDCG@5         0.038033
HR@5           0.000000
Name: 19, dtype: float64

최고 성능 (HR@5 기준):
alpha          0.000000
Precision@5    0.004567
Recall@5       0.004567
NDCG@5         0.002750
HR@5           0.000000
Name: 0, dtype: float64
